In [6]:
# Dependencies
from bs4 import BeautifulSoup
import requests
import os
import pandas as pd
from splinter import Browser

In [7]:
# Choose the executable path to driver 
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [8]:
url ="https://mars.nasa.gov/news/"
browser.visit(url)

In [9]:
# HTML Object
html = browser.html

# Parse HTML with Beautiful Soup
soup = BeautifulSoup(html, 'html.parser')
# Retrieve the latest element that contains news title and news_paragraph
news_title=soup.find('div',class_="content_title").find('a').text
news_p=soup.find('div',class_="article_teaser_body").text

# Display scrapped data 
print(news_title)
print(news_p)


Mars 2020 Rover's 7-Foot-Long Robotic Arm Installed
The main robotic arm has been installed on NASA's newest rover. When complete, the arm will enable the rover to hold and use science tools like a human geologist would.


## JPL Mars Space Images - Featured Image

In [10]:
jpl_url='https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
        
browser.visit(jpl_url)

In [11]:
#HTML Object
html_image=browser.html
soup_img=BeautifulSoup(html_image,'html.parser')
#Find the image url for the current Featured Mars Image

image_soup=soup_img.find('div',class_="carousel_items")
image_url=image_soup.find('article')['style']
featured_image_url=image_url.replace("background-image: url('","").replace("');","")            

#background-image: url('/spaceimages/images/wallpaper/PIA18903-1920x1200.jpg');
print(featured_image_url)


/spaceimages/images/wallpaper/PIA01320-1920x1200.jpg


## Mars Weather

In [12]:
# Visit Mars Weather Twitter through splinter module
weather_url = 'https://twitter.com/marswxreport?lang=en'
browser.visit(weather_url)

In [13]:
#HTML Object
html_weather=browser.html
soup_weather=BeautifulSoup(html_weather, 'html.parser')

# print(soup_weather)

# Find all elements that contain tweets
latest_tweets = soup_weather.find_all('div', class_='js-tweet-text-container')

# Retrieve all elements that contain news title in the specified range
# Look for entries that display weather related words to exclude non weather related tweets 
for tweet in latest_tweets: 
    weather_tweet = tweet.find('p').text
    if 'Sol' and 'pressure' in weather_tweet:
        print(weather_tweet)
        break
    else: 
        pass

InSight sol 206 (2019-06-25) low -103.5ºC (-154.2ºF) high -25.3ºC (-13.5ºF)
winds from the SE at 4.3 m/s (9.6 mph) gusting to 15.6 m/s (34.8 mph)
pressure at 7.60 hPapic.twitter.com/lqcwvVG0c9


## Mars Facts

In [14]:
# Visit Mars facts url 
facts_url = 'http://space-facts.com/mars/'

# Use Panda's `read_html` to parse the url
mars_facts = pd.read_html(facts_url)

# Find the mars facts DataFrame in the list of DataFrames as assign it to `mars_df`
mars_df = mars_facts[0]

# Assign the columns `['Description', 'Value']`
mars_df.columns = ['Description','Value']

# Set the index to the `Description` column without row indexing
mars_df.set_index('Description', inplace=True)

# Save html code to folder Assets
mars_df.to_html()

data = mars_df.to_dict(orient='records')  # Here's our added param..

# Display mars_df
mars_df

,Value
Description,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.42 x 10^23 kg (10.7% Earth)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.52 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-153 to 20 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


## Mars Hemispheres

In [15]:
hemisphere_url="https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(hemisphere_url)

In [16]:
#HTML Object
html_hemisphere=browser.html
soup_hemisphere=BeautifulSoup(html_hemisphere, 'html.parser')
img_soup=soup_hemisphere.find_all('div',class_='item' )
# print(img_soup)
# Create empty list for hemisphere urls 
hemisphere_image_urls = []
hemisphere_base_url="https://astrogeology.usgs.gov"
for i in img_soup:
    # Store title
    title=i.find("h3").text
    # Store link that leads to full image website
    click_url=i.find('a', class_='itemLink product-item')['href']
    browser.visit(hemisphere_base_url+click_url)
    # Parse HTML with Beautiful Soup for every individual hemisphere information website 
    full_img_soup=browser.html
   
    soup = BeautifulSoup(full_img_soup, 'html.parser')
    # Retrieve full image source 
    img_url = hemisphere_base_url+ soup.find('img', class_='wide-image')['src']
    
    # Append the retreived information into a list of dictionaries 
    hemisphere_image_urls.append({"title" : title, "img_url" : img_url})
    

# Display hemisphere_image_urls
hemisphere_image_urls
    


[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/3cdd1cbf5e0813bba925c9030d13b62e_schiaparelli_enhanced.tif_full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/ae209b4e408bb6c3e67b6af38168cf28_syrtis_major_enhanced.tif_full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/7cf2da4bf549ed01c17f206327be4db7_valles_marineris_enhanced.tif_full.jpg'}]